## Artificial Neural Networks for Handwritten Digits Classification

In this module we will develop another multilayer perceptrion, but now we will dive up in the tensorflow library. We will keep following the same project structure:

1. Loading Data
2. Exploring Data
3. Data preprocessing: Rescaling (with one hot encoding) and Validation dataset
4. Set up the model
5. Deploying the model
6. Testing our model

**NOTE:** Check the Artificial Neural Networks and the Image Classifier example before reading this notebook. There are several extremely important foundation concepts for you. **In this notebook we will focus on Tensorflow and not in the Neural Networks basic concepts.**

In [392]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os

from numpy.random import seed
seed(888)
tf.random.set_seed(404)

from time import strftime
from PIL import Image 

# Constants

X_TRAIN_PATH = '/content/drive/My Drive/digit_xtrain.csv'
X_TEST_PATH = '/content/drive/My Drive/digit_xtest.csv'
Y_TRAIN_PATH = '/content/drive/My Drive/digit_ytrain.csv'
Y_TEST_PATH = '/content/drive/My Drive/digit_ytest.csv'
LOGGING_PATH = '/content/drive/My Drive/Tensorboard_mnist_digit_logs'


IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
COLOUR_CHANNELS = 1
TOTAL_INPUTS = IMAGE_HEIGHT * IMAGE_WIDTH * COLOUR_CHANNELS

In [280]:
# 1. Loading the data
y_train_all = np.loadtxt(Y_TRAIN_PATH, dtype=int, delimiter=',')
y_test_all = np.loadtxt(Y_TEST_PATH, dtype=int, delimiter=',')

x_train_all = np.loadtxt(X_TRAIN_PATH, dtype=int, delimiter=',')
x_test_all = np.loadtxt(X_TEST_PATH, dtype=int, delimiter=',')

In [281]:
# 2. Data Exploration 
# We are working with images from handwritten digits in grayscale. Measurements: (28 x 28 x 1 - single color channel) = 784

print('y train shape:', y_train_all.shape) # 60.000 labels to go along with our x train featues
print('y test shape:', y_test_all.shape) # 10.000 labels for testing our results

print('x train shape:', x_train_all.shape) # 60.000 images examples and each example 784 values associated (size of the image)
print('x test shape:', x_test_all.shape) # 10.000 images for testing and each one has 784 values associated (size of the image)

y train shape: (60000,)
y test shape: (10000,)
x train shape: (60000, 784)
x test shape: (10000, 784)


In [282]:
# Dive in the array with the first example:
x_train_all[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [283]:
# Checking our y_train_all labels
print('First y_train label is:', y_train_all[0]) # The answer is '5' for the first example
print('Five first y_train labels are:', y_train_all[:5]) 

# Note that our array is already flattened, because we have the 784 shape. If there was not flattened the size would be 10000, 28, 28, 1.
# Good: It is easy to work
# Bad: We loose a little bit information positioning


First y_train label is: 5
Five first y_train labels are: [5 0 4 1 9]


In [284]:
# 3. Data Preprocessing

# Rescaling our data: >> we will transform our dataset to 0 to 1 scale instead 0 - 255 to make our future calculations easier.
x_train_all, x_test_all = x_train_all / 255.0 , x_test_all / 255.0

In [285]:
# One-Hot Encoding >> Transforming our data to 0 - 1 scale, in which each column is a feature and in the answer will be assgined with 0 or 1. Good for categorical data
# We will transform our sparse matrix (5, 0, 4, 1, 9...) into a full matrix (each column will be the answer containing 0 or 1)

# Example:
values = y_train_all[:5]

# Build a 2-D array with 1's and zeroes given some pattern. By defaul the diagonal is 1.
np.eye(10)[values] 

# np.eye(N ==> amount of rows, M ==> amount of columns)[index pattern] 
# Similarly if we use np.eye(10)[2] we will have a 10 x 10 matrix with '1' in the index 2 for each row. Other ones will be zero 

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [286]:
# Applying to our dataset

NR_CLASSES = 10 # 0 - 9 digit = 10 classes in our dataset.

y_train_all = np.eye(NR_CLASSES)[y_train_all]

print('y train reshaped:', y_train_all.shape) # 60.000 examples (features) + 10 columns one hot encoded

y train reshaped: (60000, 10)


In [287]:
y_test_all = np.eye(NR_CLASSES)[y_test_all]

print('y test reshaped:', y_test_all.shape) # 10.000 examples (features) + 10 columns one hot encoded

y test reshaped: (10000, 10)


In [288]:
# Creating the validation dataset

VALIDATION_SIZE = 10000 # Same size of our test set

x_validation = x_train_all[:VALIDATION_SIZE]
y_validation = y_train_all[:VALIDATION_SIZE]

x_train_xs = x_train_all[VALIDATION_SIZE:]
y_train_xs = y_train_all[VALIDATION_SIZE:]

print('x train new shape:', x_train_xs.shape)
print('y train new shape:', y_train_xs.shape)

print('x validation shape:', x_validation.shape)
print('y validation shape:', y_validation.shape)

x train new shape: (50000, 784)
y train new shape: (50000, 10)
x validation shape: (10000, 784)
y validation shape: (10000, 10)


### Tensorflow Guideline

What is a Tensor? Tensor is essentialy a way to store data in a specific structure. **Basically is a container with n-dimensions called 'ranks'**.

They follow some mathematical rules, so you can add, multiply, and make several linear algebra operations like we did with numpy. This is handy when working with Neural Networks given the calculations that we will need to use in the input and hidden layers and so on.

Differently from Scikit-Learn models and Keras frameworks, **Tensorflow allows us to develop by ourselves all steps during the construction of neural network. We can deal with the inner calculations that are abstracted with the Keras/Scikit-Learn frameworks**. With tensorflow we can define how our weights and biases will work, how the layers will behave, and **how is the FLOW between layers (that's why is called tensorflow! the flow between tensors!)** etc.

Let's use the architecture sketch below in order to build our multilayer perceptron. We will have inputs from some layer that must be multiplied to outputs for another layer (imagine input hidden or output layers for an ordinary neural network).

(IMAGE HERE)

We will define how our weights and biases will be calculated, we will need to set a matrix multiplication between the output and input. 

### Setting up our Tensorflow Graph

In [459]:
# Set up Tensorflow Graph

# Disabling eager execution for compatibility with TF 2.0  >> Change the code for TF 2.0
tf.compat.v1.disable_eager_execution()

# Creating our placeholder - tensor
X = tf.compat.v1.placeholder(tf.float32, shape=[None, TOTAL_INPUTS]) # Dimensions: None >> Examples/Samples to be used in the tensor we will define later because they will be divided in the batches, 784 features
y = tf.compat.v1.placeholder(tf.float32, shape=[None, NR_CLASSES]) # Dimensions: None , 10 = number of categories that we wanna predict

### Setting up our Neural Network Architecture

In [460]:
# Artificial Neural Network Architecture

# 1. Hyperparameters
nr_epocs = 50
learning_rate = 0.001 # or 1e-3 / 1e-4 

# 2. Layers will be set in this neural network
n_hidden1 = 512 # amount of neurons
n_hidden2 = 64 # amount of neurons
# Output layer will have 10 neurons

In [461]:
def setup_layer(input, weight_dim, bias_dim, name):

  with tf.name_scope(name):

    # A. Weights setup
    initial_w = tf.compat.v1.truncated_normal(shape=weight_dim, stddev= 0.1, seed= 42) # shape = [n_hidden2, NR_CLASSES]
    w = tf.compat.v1.Variable(initial_value=initial_w, name='W')

    # B. Biases setup
    initial_b = tf.compat.v1.constant(value=0.0, shape=bias_dim)
    b = tf.compat.v1.Variable(initial_value=initial_b, name='B')

    # C. How Layer will work
    # Layer - Inputs and Calculations
    layer_in = tf.linalg.matmul(input, w) + b # Input = results from the previous layer. 

    # Layer - Output - Activation Function
    if name == 'output':

      layer_out = tf.nn.softmax(layer_in)
    
    else: 

      layer_out = tf.nn.relu(layer_in)
    
    # Tensorboard
    tf.summary.histogram('weights', w) # Summary for weights
    tf.summary.histogram('biases', b) # Summary for weights
    
    return layer_out
    

In [462]:
# Model Architecture 

layer_1 = setup_layer(X, 
                      weight_dim=[TOTAL_INPUTS, n_hidden1], 
                      bias_dim= [n_hidden1], 
                      name='layer_1')

layer_2 = setup_layer(layer_1, 
                      weight_dim=[n_hidden1, n_hidden2], 
                      bias_dim= [n_hidden2], 
                      name='layer_2')

output = setup_layer(layer_2, 
                     weight_dim=[n_hidden2, NR_CLASSES], 
                     bias_dim= [NR_CLASSES], 
                     name='output')

model_name = f'{n_hidden1}-{n_hidden2}-Learning Rate:{learning_rate} Epochs:{nr_epocs}'


In [463]:
# Model Architecture with Dropouts
"""
layer_1 = setup_layer(X, 
                      weight_dim=[TOTAL_INPUTS, n_hidden1], 
                      bias_dim= [n_hidden1], 
                      name='layer_1')

# Tensorflow 1.x >> layer_dropout = tf.compat.v1.nn.dropout(layer_1, keep_prob=0.8, name='dropout_layer')
# Tensorflow 2.0
layer_dropout= tf.nn.dropout(layer_1, 0.8) # Parameters: Previous Layer + % of dropout 

layer_2 = setup_layer(layer_dropout, 
                      weight_dim=[n_hidden1, n_hidden2], 
                      bias_dim= [n_hidden2], 
                      name='layer_2')

output = setup_layer(layer_2, 
                     weight_dim=[n_hidden2, NR_CLASSES], 
                     bias_dim= [NR_CLASSES], 
                     name='output')

model_name = f'{n_hidden1}-{n_hidden2}-Dropout-Learning Rate:{learning_rate} Epochs:{nr_epocs}'
"""



"\nlayer_1 = setup_layer(X, \n                      weight_dim=[TOTAL_INPUTS, n_hidden1], \n                      bias_dim= [n_hidden1], \n                      name='layer_1')\n\n# Tensorflow 1.x >> layer_dropout = tf.compat.v1.nn.dropout(layer_1, keep_prob=0.8, name='dropout_layer')\n# Tensorflow 2.0\nlayer_dropout= tf.nn.dropout(layer_1, 0.8) # Parameters: Previous Layer + % of dropout \n\nlayer_2 = setup_layer(layer_dropout, \n                      weight_dim=[n_hidden1, n_hidden2], \n                      bias_dim= [n_hidden2], \n                      name='layer_2')\n\noutput = setup_layer(layer_2, \n                     weight_dim=[n_hidden2, NR_CLASSES], \n                     bias_dim= [NR_CLASSES], \n                     name='output')\n\nmodel_name = f'{n_hidden1}-{n_hidden2}-Dropout-Learning Rate:{learning_rate} Epochs:{nr_epocs}'\n"

## Step by step intuition

The function setup_layer summarizes the codepads below. 

In [464]:
                                  ############## Layer 1 Architecture ##############
"""

with tf.name_scope('hidden_1'):
######### Weights and Biases initial guess (random values). With tensorflow we can pick them within a distribuition ##############

  # A. Initial weights in the first hidden layer >> Remember they are responsible for flatting or sheeping our activation function
  # This function will follow the normal distribuition truncating outliers from tails. 
  initial_w1 = tf.compat.v1.truncated_normal(shape=[TOTAL_INPUTS, n_hidden1], # Shape = inputs and neurons in the layer
                                            stddev= 0.1, # How to pick our guesses? Deviation by 0.1
                                            seed= 42) 

  # Important: We can't display our initial weights yet. Tensorflow works by stages: 1st stage set up ; 2nd how to perform calculations >> After that we can see our objects

  # Creating a tensorflow variable, because our weights must be updated later
  w1 = tf.compat.v1.Variable(initial_value=initial_w1)



  # B. Initializing biases >> Remember they will be responsible for shifiting our activation function.
  initial_b1 = tf.compat.v1.constant(value=0.0, # initial value will be zero, in other words no biases at the beginning
                                    shape=[n_hidden1]) # shape of our biases

  # Creating the same tensorflow variable
  b1 = tf.compat.v1.Variable(initial_value=initial_b1)


  # C. Determining how the weights and biases will work in our neural network and defining the flow control between inputs/outputs

  # Layer 1 - Inputs and Calculations
  layer1_in = tf.linalg.matmul(X, w1) + b1 # INPUT LAYER + HIDDEN LAYER = Multiply our X inputs with the weights + bias in the first layer 

  # Layer 1 - Output - Activation Function
  layer1_out = tf.nn.relu(layer1_in) # HIDDEN LAYER (512 neurons hidden layer)
"""

"\n\nwith tf.name_scope('hidden_1'):\n######### Weights and Biases initial guess (random values). With tensorflow we can pick them within a distribuition ##############\n\n  # A. Initial weights in the first hidden layer >> Remember they are responsible for flatting or sheeping our activation function\n  # This function will follow the normal distribuition truncating outliers from tails. \n  initial_w1 = tf.compat.v1.truncated_normal(shape=[TOTAL_INPUTS, n_hidden1], # Shape = inputs and neurons in the layer\n          stddev= 0.1, # How to pick our guesses? Deviation by 0.1\n          seed= 42) \n\n  # Important: We can't display our initial weights yet. Tensorflow works by stages: 1st stage set up ; 2nd how to perform calculations >> After that we can see our objects\n\n  # Creating a tensorflow variable, because our weights must be updated later\n  w1 = tf.compat.v1.Variable(initial_value=initial_w1)\n\n\n\n  # B. Initializing biases >> Remember they will be responsible for shifiting

**Tensorflow Variables**: Imagine a variable like something that preserves the state of the graph. They can be updated and changed! for this reason w1 and b1 are variables. We need to initialize all variables, that's why we will assign 'global_variables_initializer()' later on.

**Constants:** Values that don't change over time

In [465]:
                                  ############## Layer 2 Architecture ##############
"""
with tf.name_scope('hidden_layer_2'):
  # A. Weights setup
  initial_w2 = tf.compat.v1.truncated_normal(shape=[n_hidden1, n_hidden2], # Shape = nr of neurons from the previous layer and neurons in the current layer
                                            stddev= 0.1, # How to pick our guesses? Deviation by 0.1
                                            seed= 42) 

  w2 = tf.compat.v1.Variable(initial_value=initial_w2)

  # B. Biases setup
  initial_b2 = tf.compat.v1.constant(value=0.0, # initial value will be zero, in other words no biases at the beginning
                                    shape=[n_hidden2]) # shape of our biases

  b2 = tf.compat.v1.Variable(initial_value=initial_b2)


  # C. How Layer 2 will work
  # Layer 2 - Inputs and Calculations
  layer2_in = tf.linalg.matmul(layer1_out, w2) + b2 # 1st HIDDEN LAYER OUTPUT (512 neurons) + 2nd HIDDEN LAYER (64 neurons) = Multiply our X inputs with the weights + bias in the previous layer 

  # Layer 2 - Output - Activation Function
  layer2_out = tf.nn.relu(layer2_in) # 2nd HIDDEN LAYER (64 neurons hidden layer)
"""

"\nwith tf.name_scope('hidden_layer_2'):\n  # A. Weights setup\n  initial_w2 = tf.compat.v1.truncated_normal(shape=[n_hidden1, n_hidden2], # Shape = nr of neurons from the previous layer and neurons in the current layer\n          stddev= 0.1, # How to pick our guesses? Deviation by 0.1\n          seed= 42) \n\n  w2 = tf.compat.v1.Variable(initial_value=initial_w2)\n\n  # B. Biases setup\n  initial_b2 = tf.compat.v1.constant(value=0.0, # initial value will be zero, in other words no biases at the beginning\n  shape=[n_hidden2]) # shape of our biases\n\n  b2 = tf.compat.v1.Variable(initial_value=initial_b2)\n\n\n  # C. How Layer 2 will work\n  # Layer 2 - Inputs and Calculations\n  layer2_in = tf.linalg.matmul(layer1_out, w2) + b2 # 1st HIDDEN LAYER OUTPUT (512 neurons) + 2nd HIDDEN LAYER (64 neurons) = Multiply our X inputs with the weights + bias in the previous layer \n\n  # Layer 2 - Output - Activation Function\n  layer2_out = tf.nn.relu(layer2_in) # 2nd HIDDEN LAYER (64 neurons hi

In [466]:
                                  ############## Output Layer Architecture ##############
"""
# Name Scope: Helps us to organize our code. All the weights, biases, calculations are in the same context from 'Output Layer'
# Our Tensorflow Graph will summurize all these calculations into just one scope. 
with tf.name_scope('output_layer'):

  # A. Weights setup
  initial_w3 = tf.compat.v1.truncated_normal(shape=[n_hidden2, NR_CLASSES], # Shape = nr of neurons from the previous layer , neurons in the current layer
                                            stddev= 0.1, # How to pick our guesses? Deviation by 0.1
                                            seed= 42) 

  w3 = tf.compat.v1.Variable(initial_value=initial_w3)

  # B. Biases setup
  initial_b3 = tf.compat.v1.constant(value=0.0, # initial value will be zero, in other words no biases at the beginning
                                    shape=[NR_CLASSES]) # shape of our biases given the current output

  b3 = tf.compat.v1.Variable(initial_value=initial_b3)


  # C. How Layer 2 will work
  # Layer 2 - Inputs and Calculations
  layer3_in = tf.linalg.matmul(layer2_out, w3) + b3 # 1st HIDDEN LAYER OUTPUT (512 neurons) + 2nd HIDDEN LAYER (64 neurons) = Multiply our X inputs with the weights + bias in the previous layer 

  # Layer 2 - Output - Activation Function
  output = tf.nn.softmax(layer3_in) # OUTPUT LAYER (10 neurons) / Softmax = Probabilistic answer
"""

"\n# Name Scope: Helps us to organize our code. All the weights, biases, calculations are in the same context from 'Output Layer'\n# Our Tensorflow Graph will summurize all these calculations into just one scope. \nwith tf.name_scope('output_layer'):\n\n  # A. Weights setup\n  initial_w3 = tf.compat.v1.truncated_normal(shape=[n_hidden2, NR_CLASSES], # Shape = nr of neurons from the previous layer , neurons in the current layer\n          stddev= 0.1, # How to pick our guesses? Deviation by 0.1\n          seed= 42) \n\n  w3 = tf.compat.v1.Variable(initial_value=initial_w3)\n\n  # B. Biases setup\n  initial_b3 = tf.compat.v1.constant(value=0.0, # initial value will be zero, in other words no biases at the beginning\n  shape=[NR_CLASSES]) # shape of our biases given the current output\n\n  b3 = tf.compat.v1.Variable(initial_value=initial_b3)\n\n\n  # C. How Layer 2 will work\n  # Layer 2 - Inputs and Calculations\n  layer3_in = tf.linalg.matmul(layer2_out, w3) + b3 # 1st HIDDEN LAYER OUTP

### Note that our code before is written a little bit repeatedly, why not define a function.

### Tensorboard Setup

In [467]:
## Tensorboard Setup

folder_name = f'Model {model_name} at {strftime("%H:%M")}' # For each model creates a specific folder
directory = os.path.join(LOGGING_PATH, folder_name)

try:

  os.makedirs(directory)

except OSError as e:

  print(e)

else:

  print("Directory created successfully")

# Tensorboard needs to create summaries from our metrics. For this reason we will need to bound our metrics with the tensorboard

Directory created successfully


In [468]:
                                  ############## Loss, Optimizer and Metrics ##############

# A. Loss
# Cross Entropy Loss >> Given the classification task is the best function to calculate our loss.
# logits = Outputs from our Output Layer 
# labels = Acctual Labels (y values)

# Tensorflow 2.0 >> loss = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=output)

with tf.name_scope('loss_calc'):
  # We will calculate our model with Batches, for this reason we need to take the avg
  loss = tf.reduce_mean(tf.compat.v1.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=output))        


# B. Optimizer
with tf.name_scope('optimizer'):
  # Set up our optimizer with our learning rate (by default is 1)
  optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate= learning_rate) 

  # Update the values from the previous layers (Backpropagation) by minimizing our loss (Note: If you use uphill approach with SGD you'll need to adapt here)
  train_step = optimizer.minimize(loss)


# C. Metrics - Accuracy
with tf.name_scope('accuracy_calc'):
  correct_prediction = tf.compat.v1.equal(tf.argmax(output, axis=1), tf.argmax(y, axis=1)) # We need to adapt given that the one hot encoded our outputs!
  accuraccy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) # get the mean given that we are working with batches!

In [469]:
with tf.name_scope('performance'):
  # Adding 'accuracy' metric to tensorboard
  tf.compat.v1.summary.scalar('accuraccy', accuraccy)
  tf.compat.v1.summary.scalar('loss', loss)

In [470]:
# Checking images with Tensorboard

with tf.name_scope('show_image'):

  # 1º we need to reshape for a 4-D tensor
  x_image = tf.reshape(X,[-1, 28, 28, 1]) # 28 x 28 x 1 + 1 new dimension (4-D) dummy
  tf.compat.v1.summary.image('image_put', x_image, max_outputs=4)

### Training our model (Sessions, Batching and Training)

Tensorflow works with Session objects, that are responsible for encapsulate the operations and calculations. 

In the Session that we start all variables, and manage all the placeholders and flow from our code/graph. Placeholders must be fed with data during the session, for this reason we placed the dictionary in our batch function!

In [471]:
# Sessions, Batching and Training

# A. Sessions - Defining our session and initializing all variables previously set up
sess = tf.compat.v1.Session()

# Initialize global variables
init = tf.compat.v1.global_variables_initializer()

# Tensorboard step
merged_summary = tf.compat.v1.summary.merge_all() # Merge all summaries like a global collector
train_writer = tf.compat.v1.summary.FileWriter(directory + '/train') # File Writer: Manager to write down data and set the local to save our training summary files
train_writer.add_graph(sess.graph) # All calculations from Tensorflow are graphs. so we need to redirect from Tensorboard our session Graph

# Validation writer! We don't need to append the graph because they use the same session
validation_writer = tf.compat.v1.summary.FileWriter(directory + '/validation')



# Binding our variables to the session 
sess.run(init)

# After running we can check our previous variables! w1, b1, etc.

In [472]:
# Checking our starting weights
#w1.eval(sess)

In [473]:
# Checking our starting biases
#b1.eval(sess)

In [474]:
# Batching the Data >> Split up our training dataset into small batches for better performance!

size_of_batch = 1000

num_examples = y_train_xs.shape[0]
nr_iterations = int(num_examples / size_of_batch) # 50 

index_in_epoch = 0 # Keep tracking that the 1st epoch will start with the index 0 - 999. 2nd batch 1000 - 1999 and so onsiz

In [475]:
# Function to transfer the current batch to the next batch
def next_batch(batch_size, data, labels):

  global num_examples # global collects the variable that is outside this context
  global index_in_epoch

  # Batch Start Controller
  start = index_in_epoch
  
  # Updates our index in epoch with the size batch size
  index_in_epoch += batch_size

  if index_in_epoch > num_examples:

    #start next epoch
    start = 0
    index_in_epoch = batch_size

  # Batch End Controller
  end = index_in_epoch

  return data[start:end], labels[start:end] # Features and Labels


In [476]:
# Training our model (Loop)

for epoch in range(nr_epocs):

  for i in range(nr_iterations):

    batch_x, batch_y = next_batch(batch_size= size_of_batch,
                                  data= x_train_xs,
                                  labels= y_train_xs)
    
    # Feed dictionary >> Used to feed our session with data
    feed_dictionary = {X: batch_x, y: batch_y} # X and y where defined in the tensorflow graph set up! Be minded that our placeholder and our data must have the same shape

    # Execute our train step (Optimizer = Reducing Loss) given our data dictionary
    sess.run(train_step, feed_dict= feed_dictionary)

    # Accuracy: Fetches the calculated accuracy (like a callback)
    #batch_accuracy = sess.run(fetches=[accuraccy], feed_dict= feed_dictionary) # add our merged_summary to create a summary each epoch

  
  # Tensorboard 
  s, batch_accuracy = sess.run(fetches=[merged_summary, accuraccy], feed_dict= feed_dictionary) # >>> Tensorboard
  train_writer.add_summary(s, epoch) # Each epoch saves the summary 's'

  print(f'Epoch {epoch} \t| Training accuracy = {batch_accuracy}')

  # Tensorboard - Validation 
  summary = sess.run(fetches=merged_summary, feed_dict={X:x_validation, y:y_validation})
  validation_writer.add_summary(summary, epoch)

print('Training finished')

Epoch 0 	| Training accuracy = 0.859000027179718
Epoch 1 	| Training accuracy = 0.8669999837875366
Epoch 2 	| Training accuracy = 0.8709999918937683
Epoch 3 	| Training accuracy = 0.875
Epoch 4 	| Training accuracy = 0.8759999871253967
Epoch 5 	| Training accuracy = 0.9620000123977661
Epoch 6 	| Training accuracy = 0.9829999804496765
Epoch 7 	| Training accuracy = 0.9860000014305115
Epoch 8 	| Training accuracy = 0.984000027179718
Epoch 9 	| Training accuracy = 0.9869999885559082
Epoch 10 	| Training accuracy = 0.9869999885559082
Epoch 11 	| Training accuracy = 0.9879999756813049
Epoch 12 	| Training accuracy = 0.9890000224113464
Epoch 13 	| Training accuracy = 0.9900000095367432
Epoch 14 	| Training accuracy = 0.9890000224113464
Epoch 15 	| Training accuracy = 0.9900000095367432
Epoch 16 	| Training accuracy = 0.9909999966621399
Epoch 17 	| Training accuracy = 0.9909999966621399
Epoch 18 	| Training accuracy = 0.9909999966621399
Epoch 19 	| Training accuracy = 0.9900000095367432
Epoch

## Making Predictions!

In [477]:
# Loading our test image!
img = Image.open('/content/drive/My Drive/test_img.png')
img

# We will need to adjust our image given our trained formatting
# Inverted grayscale
# Flattening 

bw_image = img.convert(mode='L') # Grayscale reversed
img_array = np.invert(bw_image) 
print('Current shape:', img_array.shape) # 28 x 28 --> We don't have the channel anymore given the grayscale conversion

test_image = img_array.ravel() # Flattening array
print('Image shape after flattening:', test_image.shape)

Current shape: (28, 28)
Image shape after flattening: (784,)


In [478]:
# Calling our session to make the predictions

# fetches => Takes the biggest row from our output
# feed_dict => We just need to provide our image
prediction = sess.run(fetches=tf.argmax(output, axis=1), feed_dict={X:[test_image]}) 

print('Prediction for test image is:', prediction)

Prediction for test image is: [2]


## Testing and Evaltuation


In [479]:
# Checking the accuracu with our X test and y test datasets

test_accuracy = sess.run(fetches=accuraccy, feed_dict={X: x_test_all, y: y_test_all})
print(f'Accuracy on the test set is {test_accuracy:0.2%}')

Accuracy on the test set is 97.89%


In [480]:
# Reseting for the next run. These will 'clean' our calculations preventing anomalities
train_writer.close()
validation_writer.close()
sess.close()
tf.compat.v1.reset_default_graph()